# Data processing course assignments
---

In [1]:
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

In [ ]:
#conf = SparkConf().setAppName('Assignment').setMaster('local')
#sc = SparkContext(conf=conf)

Introducing `SparkSession`, it is a combination of the previous versions `SparkConf`, `SparkContext`, `SQLContext`, and `HiveContext`.

In [2]:
spark = SparkSession \
    .builder \
    .appName("Python Spark") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

## Ejercicio 1.
#### Leer el archivo data/containers.csv y contar el número de líneas.
---

Obteniendo los datos del archivo csv.

In [3]:
pwd

'/home/jovyan/work/PySpark-master'

In [4]:
ls

Assignments.ipynb*  pysparkTextMining.ipynb*  spark-warehouse/
PySpark.ipynb*      README.md*


In [9]:
path = "/home/jovyan/work/data_processing_course-master/assignments/data/containers.csv"
csvdata = spark.read.format("csv").option("header", "true").load(path)
#spark.csv.load('/home/jovyan/work/data_processing_course-master/assignments/data/containers.csv')

In [10]:
# Comprobando obtención de los datos.
csvdata.take(2)

[Row(ship_imo;ship_name;country;departure;container_id;container_type;container_group;net_weight;gross_weight;owner;declared;contact;customs_ok='AMC1861710;Jayden;BD;201602183;FCUK1755843;4960;28VH;44804866.62;2240243.33;Streich-Wilkinson;Music'),
 Row(ship_imo;ship_name;country;departure;container_id;container_type;container_group;net_weight;gross_weight;owner;declared;contact;customs_ok='POG1615575;Lake Eribertoland;CR;2016021611;PDXW1549639;28VH;8888;16681047.32;500431.42;Senger and Sons;Movies & Jewelery;cindy.dubuque@roberts.org;true')]

Contando el número de líneas del archivo csv.

In [11]:
csvdata.count()

613

## Ejercicio 2.
#### Leer el archivo data/containers.csv y filtrar aquellos contenedores cuyo ship_imo es DEJ1128330 y el grupo del contenedor es 22P1. Guardar los resultados en un archivo de texto en resultados/resutado_2.
---

In [28]:
print(type(csvdata))
datardd = csvdata.rdd
print(type(datardd))

datasplit = datardd.map(lambda row: row[0].split(";"))

datasplit.collect()

<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.rdd.RDD'>


[['AMC1861710',
  'Jayden',
  'BD',
  '201602183',
  'FCUK1755843',
  '4960',
  '28VH',
  '44804866.62',
  '2240243.33',
  'Streich-Wilkinson',
  'Music'],
 ['POG1615575',
  'Lake Eribertoland',
  'CR',
  '2016021611',
  'PDXW1549639',
  '28VH',
  '8888',
  '16681047.32',
  '500431.42',
  'Senger and Sons',
  'Movies & Jewelery',
  'cindy.dubuque@roberts.org',
  'true'],
 ['SQH1155999',
  'Aileen',
  'GN',
  '201602191',
  'PLKO1661930',
  'L2R3',
  'L5R1',
  '69102632.38',
  '3455131.62',
  'Thompson-Kautzer',
  'Movies & Jewelery',
  'otha@granteichmann.io',
  'true'],
 ['JCI1797526',
  'Herminio',
  'KH',
  '2016021610',
  'BXMT1827488',
  '44R1',
  'L5G1',
  '4367327.08',
  '436732.71',
  'Herzog'],
 ['MBV1836745',
  'Port Guiseppeburgh',
  'NO',
  '2016021915',
  'JYIE1892741',
  '2161',
  '28VH',
  '4663065.05',
  '139891.95',
  'Rath-Schowalter',
  'Kids'],
 ['GYR1192020',
  'Emardland',
  'RU',
  '201602156',
  'LARQ1499256',
  '2167',
  '22R1',
  '130500887.93',
  '13050088.79

In [29]:
# datafiltered = datasplit.filter(lambda row: row == '22P1')
datasplit.map(lambda row: row[0]).take(10)

['AMC1861710',
 'POG1615575',
 'SQH1155999',
 'JCI1797526',
 'MBV1836745',
 'GYR1192020',
 'GLV1922612',
 'NLH1771681',
 'FUS1202266',
 'GLV1922612']

In [30]:
datasplit.filter(lambda row: row[6] == '22P1' and row[0] == 'DEJ1128330').count()

2

In [31]:
datasplit.filter(lambda row: row[6] == '22P1' and row[0] == 'DEJ1128330').take(2)

[['DEJ1128330',
  'Tiara',
  'GP',
  '2016021818',
  'GYFD1228113',
  '20PF',
  '22P1',
  '51503716.88',
  '5150371.69',
  'Armstrong-Goldner',
  'Automotive'],
 ['DEJ1128330',
  'Tiara',
  'GP',
  '2016021818',
  'MBPF1909627',
  '24H2',
  '22P1',
  '37266600.88',
  '1863330.04',
  'Lehner-Hamill',
  'Jewelery']]

Guardando los datos como archivo de texto

In [36]:
x1 = datasplit.filter(lambda row: row[6] == '22P1' and row[0] == 'DEJ1128330')
x1.saveAsTextFile('data_processing_course-master/assignments/resultados/resultado_2')

## Ejercicio 3.
#### Leer el archivo data/containers.csv y convertir a formato Parquet. Recuerda que puedes hacer uso de la funcion parse_container en helpers.py tal y como vimos en clase. Guarda los resultados en resultados/resultado_3.
---

Using `SparkSession` to create a Dataframe from a csv. I am using a Dataframe instead of a RDD, because are RDDs are schemaless, and Dataframes are close to Parquet files structure.

In [33]:
data3 = spark.read.csv(path, header=True, sep=';') #Usamos el mismo path creado previamente

In [34]:
data3.show(3)

+----------+-----------------+-------+----------+------------+--------------+---------------+-----------+------------+-----------------+--------------------+--------------------+----------+
|  ship_imo|        ship_name|country| departure|container_id|container_type|container_group| net_weight|gross_weight|            owner|            declared|             contact|customs_ok|
+----------+-----------------+-------+----------+------------+--------------+---------------+-----------+------------+-----------------+--------------------+--------------------+----------+
|AMC1861710|           Jayden|     BD| 201602183| FCUK1755843|          4960|           28VH|44804866.62|  2240243.33|Streich-Wilkinson|Music, Tools, Aut...|octavia@stammbedn...|      true|
|POG1615575|Lake Eribertoland|     CR|2016021611| PDXW1549639|          28VH|           8888|16681047.32|   500431.42|  Senger and Sons|   Movies & Jewelery|cindy.dubuque@rob...|      true|
|SQH1155999|           Aileen|     GN| 201602191| 

In [30]:
cd data_processing_course-master/assignments/resultados

/home/jovyan/work/data_processing_course-master/assignments/resultados


In [36]:
data3.write.parquet("data_processing_course-master/assignments/resultados/resultado_3")

## Ejercicio 4.
#### Lee el archivo de Parquet guardado en el ejercicio 3 y filtra los barcos que tienen al menos un contenedor donde la columna customs_ok es igual a false. Extrae una lista con los identificadores de barco, ship_imo, sin duplicados y ordenados alfabéticamente, en formato json.
---

In [37]:
data4 = spark.read.parquet("data_processing_course-master/assignments/resultados/resultado_3.parquet")

## Ejercicio 5.
#### Crea una UDF para validar el código de identificación del contenedor container_id. Para simplificar la validación, daremos como válidos aquellos códigos compuestos de 3 letras para el propietario, 1 letra para la categoría, 6 números y 1 dígito de control. Devuelve un DataFrame con los campos: ship_imo, container_id, propietario, categoria, numero_serie y digito_control.
---

## Ejercicio 6. 
#### Extrae una lista con peso total de cada barco, net_weight, sumando cada contenedor y agrupado por los campos ship_imo y container_group. Devuelve un DataFrame con la siguiente estructura: ship_imo, ship_name, container, total_net_weight.
---

## Ejercicio 7.
#### Guarda los resultados del ejercicio anterior en formato Parquet.
---

## Ejercicio 8.
#### ¿En qué casos crees que es más eficiente utilizar formatos como Parquet? ¿Existe alguna desventaja frente a formatos de texto como CSV?
---

## Ejercicio 9.
#### ¿Es posible procesar XML mediante Spark? ¿Existe alguna restricción por la cual no sea eficiente procesar un único archivo en multiples nodos? ¿Se te ocurre alguna posible solución para trocear archivos suficientemente grandes? ¿Existe la misma problemática con otros formatos de texto como JSON?
---

## Ejercicio 10.
#### Spark SQL tiene una función denominada avg que se utiliza para calcular el promedio de un conjunto de valores ¿Por qué los autores han creado esta función en lugar de usar el API estándar de Python o Scala?
---